# 1) Retrieve PPE Data #

Code for retrieving data from the PPE output files. <br>
<br

Inputs:<br> 
The Full CLM-PFT-PPE dataset availible at [<br>]
pft_id - a helper dataset for mapping patches to their correct PF<br>T
nonrepresentative_pftarea.nc - the land area for each PF<br>T<br>

Ouputs<br>:
Metrics.nc - contains select output variables used for analysi<br>s

## Load in Packages ##

In [1]:
import importlib.util
import os
import cartopy.crs as ccrs
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from analysis_wave import *

In [13]:
repo_dir = os.getcwd()

## Set up parallel processing ##

In [5]:
# Setup Cluster for parallel processing
# only run this cell once!
import dask
from dask.distributed import Client
from dask_jobqueue import PBSCluster

ncores = 1
nmem = "10GB"
cluster = PBSCluster(
    cores=ncores,  # The number of cores you want
    memory=nmem,  # Amount of memory
    processes=1,  # How many processes
    queue="casper",  # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory="$TMPDIR",  # Use your local directory
    resource_spec="select=1:ncpus=" + str(ncores) + ":mem=" + nmem,  # Specify resources
    account="",  # Input your project ID here
    walltime="00:30:00",  # Amount of wall time
    interface="mgt",  # Interface to use
)

# cluster.worker_options = {'--no-nanny': None}

# Scale up
cluster.scale(30)

# Setup your client
client = Client(cluster)

In [9]:
client

<Client: 'tcp://10.18.206.63:46617' processes=30 threads=30, memory=279.30 GiB>

## Load up the model output ##

In [8]:
%%time
htape = "h1"  # (h1 is pft level)
yr0 = 1985
yr1 = 2015
ds_h1 = get_exp("wave1", tape=htape, yy=(yr0, yr1))

CPU times: user 47.1 s, sys: 1.77 s, total: 48.9 s
Wall time: 1min 11s


In [9]:
ds_h1

<xarray.Dataset>
Dimensions:             (levgrnd: 25, levsoi: 20, levlak: 10, levdcmp: 25,
                         time: 360, ens: 501, hist_interval: 2, lon: 144,
                         lat: 96, gridcell: 400, landunit: 1267, column: 2974,
                         pft: 8574, nvegwcs: 4, param: 0, biome_id: 10,
                         pft_id: 17)
Coordinates:
  * levgrnd             (levgrnd) float32 0.01 0.04 0.09 ... 19.48 28.87 42.0
  * levsoi              (levsoi) float32 0.01 0.04 0.09 0.16 ... 5.95 6.94 8.03
  * levlak              (levlak) float32 0.05 0.6 2.1 4.6 ... 25.6 34.33 44.78
  * levdcmp             (levdcmp) float32 0.01 0.04 0.09 ... 19.48 28.87 42.0
  * time                (time) object 1985-01-01 00:00:00 ... 2014-12-01 00:0...
  * lon                 (lon) float32 0.0 2.5 5.0 7.5 ... 352.5 355.0 357.5
  * lat                 (lat) float32 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * pft                 (pft) float64 0.0 1.0 2.0 3.0 4.0 ... 0.0 0.0 0.0 0.0
  * gridcell            (gridcell) int64 0 1 2 3 4 5 ... 394 395 396 397 398 399
  * biome_id            (biome_id) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: ens, hist_interval, landunit, column, nvegwcs,
                                param, pft_id
Data variables: (12/94)
    mcdate              (ens, time) int32 dask.array<chunksize=(1, 120), meta=np.ndarray>
    mcsec               (ens, time) int32 dask.array<chunksize=(1, 120), meta=np.ndarray>
    mdcur               (ens, time) int32 dask.array<chunksize=(1, 120), meta=np.ndarray>
    mscur               (ens, time) int32 dask.array<chunksize=(1, 120), meta=np.ndarray>
    nstep               (ens, time) int32 dask.array<chunksize=(1, 120), meta=np.ndarray>
    time_bounds         (ens, time, hist_interval) object dask.array<chunksize=(1, 120, 2), meta=np.ndarray>
    ...                  ...
    PREC                (time, gridcell) float32 dask.array<chunksize=(120, 400), meta=np.ndarray>
    VPD                 (time, gridcell) float32 0.2464 0.2002 ... 0.001034
    VP                  (time, gridcell) float32 1.038 1.039 ... 0.0276 0.06958
    biome               (gridcell) float64 ...
    biome_name          (biome_id) object ...
    pft_name            (pft_id) <U35 'not_vegetated' ... 'c3_irrigated'
Attributes: (12/40)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 02/22/24 17:34:50
    source:                                    Community Terrestrial Systems ...
    hostname:                                  derecho
    ...                                        ...
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          month_1
    Time_constant_3Dvars_filename:             ./PPEn14_transient_wave10001.c...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...

## Post processes the output metrics of interest ##

In [12]:
# select output metrics: GPP, CVeg, T, and the Stress factor (B):
Bmin = ds_h1.BTRANMN.compute()
T = ds_h1.FCTR.compute()
GPP = ds_h1.GPP.compute()
CVeg = ds_h1.TOTVEGC.compute()

#select a helper array of the 1-d patch to pftindex mapping array
pft_id = xr.open_dataset(repo_dir+'/utils/pft_id.nc').pfts1d_gi

#open a helper array of the land area of every patch
la = xr.open_dataset(repo_dir+'/nonrepresentative_pftarea.nc').lapft.sel(year=slice(1985, 2014))
years = GPP['time'].dt.year
la = la.reset_index('pft').reindex_like(pft_id)
la = la.sel(year=years).drop_vars('year')  # replaces year with time
la = la.assign_coords(time=GPP.time)

In [ ]:
#convert from flux densities to fluxes
GPP_la = GPP * la * 1000 * 1000
CVeg_la = Cveg * la * 1000 * 1000
T_la = T * la * 1000 * 1000

## Clean up code dimenstions from patches to gridcell/pft ##

In [20]:
pftnames = np.asarray(['not_vegetated', 'needleleaf_evergreen_temperate_tree',
       'needleleaf_evergreen_boreal_tree', 'needleleaf_deciduous_boreal_tree',
       'broadleaf_evergreen_tropical_tree',
       'broadleaf_evergreen_temperate_tree',
       'broadleaf_deciduous_tropical_tree',
       'broadleaf_deciduous_temperate_tree', 'broadleaf_deciduous_boreal_tree',
       'broadleaf_evergreen_shrub', 'broadleaf_deciduous_temperate_shrub',
       'broadleaf_deciduous_boreal_shrub', 'c3_arctic_grass',
       'c3_non-arctic_grass', 'c4_grass', 'c3_crop'], dtype=object)



def create_gridcell_ds(data, gridcell_value, varname):
    sub = data.where(data.gridcellid == gridcell_value, drop = True)
    var = varname
    gsub = sub[var].groupby(sub.pft).sum()
    ds = xr.Dataset()
    for i, pft_name in enumerate(pftnames):
        ds[pft_name] = gsub.isel(pft=i).drop_vars('pft')
    ds = ds.expand_dims({'gridcell': [gridcell_value]})
    if gridcell_value % 100 == 0:
        print(f'finished {gridcell_value}')
    return ds

def pft_to_gridcell(data, varname):
    data = data.to_dataset()
    data['gridcellid'] = pft_id.drop_indexes('pft')
    data = data.drop_indexes('pft').reset_coords('pft').rename_dims({'pft':'patch'})
    data_list = [create_gridcell_ds(data, i, varname) for i in range(1, 401)]
    ds = xr.concat(data_list, dim='gridcell')
    ds = xr.concat([ds[var] for var in pftnames], dim='pft')
    ds = ds.assign_coords(pft=pftnames).to_dataset(name=varname)
    ds['gridcell'] = ds['gridcell']-1
    return ds

In [81]:
GPP_postprocessed = pft_to_gridcell(GPP, 'GPP')
Bmin_postprocessed = pft_to_gridcell(Bmin, 'BTRANMN')
CVeg_postprocessed = pft_to_gridcell(CVeg, 'TOTVEGC')
T_postprocessed = pft_to_gridcell(T, 'FCTR')

finished 100
finished 200
finished 300
finished 400


In [88]:
Metrics = xr.merge([GPP_postprocessed, T_postprocessed, Bmin_postprocessed, CVeg_postprocessed]).isel(ens = slice(1,501)).assign_coords(ens = range(1,501))
Metrics.to_netcdf(repo_dir+'/input/Metrics.nc')